In [5]:
import librosa
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [10]:

import os
base_dir = "/Users/ankitpokhrel/Desktop/infant_cry_detection/backend/model/donateacry_corpus"
for dirname, _, filenames in os.walk(base_dir):
    for filename in filenames:
        print(os.path.join(dirname, filename))


/Users/ankitpokhrel/Desktop/infant_cry_detection/backend/model/donateacry_corpus/.DS_Store
/Users/ankitpokhrel/Desktop/infant_cry_detection/backend/model/donateacry_corpus/belly_pain/999bf14b-e417-4b44-b746-9253f81efe38-1430974001343-1.7-m-26-bp.wav
/Users/ankitpokhrel/Desktop/infant_cry_detection/backend/model/donateacry_corpus/belly_pain/C421C6FE-DFEE-4080-8AEA-848E7CE4756B-1430548333-1.0-f-26-bp.wav
/Users/ankitpokhrel/Desktop/infant_cry_detection/backend/model/donateacry_corpus/belly_pain/d6cda191-4962-4308-9a36-46d5648a95ed-1431159262344-1.7-m-04-bp.wav
/Users/ankitpokhrel/Desktop/infant_cry_detection/backend/model/donateacry_corpus/belly_pain/643D64AD-B711-469A-AF69-55C0D5D3E30F-1430138524-1.0-m-72-bp.wav
/Users/ankitpokhrel/Desktop/infant_cry_detection/backend/model/donateacry_corpus/belly_pain/643D64AD-B711-469A-AF69-55C0D5D3E30F-1430138647-1.0-m-72-bp.wav
/Users/ankitpokhrel/Desktop/infant_cry_detection/backend/model/donateacry_corpus/belly_pain/643D64AD-B711-469A-AF69-55C0D5D

In [11]:
base_dir = "/Users/ankitpokhrel/Desktop/infant_cry_detection/backend/model/donateacry_corpus"
data = []

for root, _, files in os.walk(base_dir):
    for file in files:
        if file.endswith(".wav"):  
            file_path = os.path.join(root, file)  
            label = os.path.basename(root) 
            data.append([file_path, label])

df = pd.DataFrame(data, columns=["audio_link", "label"])

csv_path = "/Users/ankitpokhrel/Desktop/infant_cry_detection/backend/model/infant_cry_data.csv"
df.to_csv(csv_path, index=False)

print(f"CSV file saved to: {csv_path}")

CSV file saved to: /Users/ankitpokhrel/Desktop/infant_cry_detection/backend/model/infant_cry_data.csv


In [12]:
dat = pd.read_csv("infant_cry_data.csv")
dat.sample(5)

,audio_link,label
411,/Users/ankitpokhrel/Desktop/infant_cry_detecti...,hungry
366,/Users/ankitpokhrel/Desktop/infant_cry_detecti...,hungry
373,/Users/ankitpokhrel/Desktop/infant_cry_detecti...,hungry
379,/Users/ankitpokhrel/Desktop/infant_cry_detecti...,hungry
380,/Users/ankitpokhrel/Desktop/infant_cry_detecti...,hungry


In [13]:
dat['audio_link']

0      /Users/ankitpokhrel/Desktop/infant_cry_detecti...
1      /Users/ankitpokhrel/Desktop/infant_cry_detecti...
2      /Users/ankitpokhrel/Desktop/infant_cry_detecti...
3      /Users/ankitpokhrel/Desktop/infant_cry_detecti...
4      /Users/ankitpokhrel/Desktop/infant_cry_detecti...
                             ...                        
452    /Users/ankitpokhrel/Desktop/infant_cry_detecti...
453    /Users/ankitpokhrel/Desktop/infant_cry_detecti...
454    /Users/ankitpokhrel/Desktop/infant_cry_detecti...
455    /Users/ankitpokhrel/Desktop/infant_cry_detecti...
456    /Users/ankitpokhrel/Desktop/infant_cry_detecti...
Name: audio_link, Length: 457, dtype: object

In [15]:
def process_audio_csv(input_csv, output_csv, n_mfcc=13):
    try:
        df = pd.read_csv(input_csv)
        if "audio_link" not in df.columns:
            print("Error: 'audio_link' column not found in CSV!")
            return
        feature_columns = [f"MFCC_{i+1}" for i in range(n_mfcc)] + \
                          ["Spectral_Centroid", "Zero_Crossing_Rate"] + \
                          [f"Chroma_{i+1}" for i in range(12)]
        for col in feature_columns:
            df[col] = None
        for i, row in df.iterrows():
            audio_path = row["audio_link"]
            try:
                y, sr = librosa.load(audio_path, sr=None)
                mfccs = np.mean(librosa.feature.mfcc(y=y, sr=sr, n_mfcc=n_mfcc), axis=1)
                spectral_centroid = np.mean(librosa.feature.spectral_centroid(y=y, sr=sr), axis=1)
                zcr = np.mean(librosa.feature.zero_crossing_rate(y), axis=1)
                chroma = np.mean(librosa.feature.chroma_stft(y=y, sr=sr), axis=1)
                features = np.concatenate([
                    mfccs, spectral_centroid.ravel(), zcr.ravel(), chroma
                ])
                df.loc[i, feature_columns] = features

                print(f"Processed: {audio_path}")

            except Exception as e:
                print(f"Error processing {audio_path}: {e}")
        df.to_csv(output_csv, index=False)
        print(f"Feature extraction complete. Saved to {output_csv}")
    except Exception as e:
        print(f"Error: {e}")
process_audio_csv("infant_cry_data.csv", "final.csv")

Processed: /Users/ankitpokhrel/Desktop/infant_cry_detection/backend/model/donateacry_corpus/belly_pain/999bf14b-e417-4b44-b746-9253f81efe38-1430974001343-1.7-m-26-bp.wav
Processed: /Users/ankitpokhrel/Desktop/infant_cry_detection/backend/model/donateacry_corpus/belly_pain/C421C6FE-DFEE-4080-8AEA-848E7CE4756B-1430548333-1.0-f-26-bp.wav
Processed: /Users/ankitpokhrel/Desktop/infant_cry_detection/backend/model/donateacry_corpus/belly_pain/d6cda191-4962-4308-9a36-46d5648a95ed-1431159262344-1.7-m-04-bp.wav
Processed: /Users/ankitpokhrel/Desktop/infant_cry_detection/backend/model/donateacry_corpus/belly_pain/643D64AD-B711-469A-AF69-55C0D5D3E30F-1430138524-1.0-m-72-bp.wav
Processed: /Users/ankitpokhrel/Desktop/infant_cry_detection/backend/model/donateacry_corpus/belly_pain/643D64AD-B711-469A-AF69-55C0D5D3E30F-1430138647-1.0-m-72-bp.wav
Processed: /Users/ankitpokhrel/Desktop/infant_cry_detection/backend/model/donateacry_corpus/belly_pain/643D64AD-B711-469A-AF69-55C0D5D3E30F-1430138495-1.0-m-72

In [16]:
final_df = pd.read_csv("final.csv")

In [17]:
final_df.sample(5)

,audio_link,label,MFCC_1,MFCC_2,MFCC_3,MFCC_4,MFCC_5,MFCC_6,MFCC_7,MFCC_8,...,Chroma_3,Chroma_4,Chroma_5,Chroma_6,Chroma_7,Chroma_8,Chroma_9,Chroma_10,Chroma_11,Chroma_12
166,/Users/ankitpokhrel/Desktop/infant_cry_detecti...,hungry,-330.828278,32.888687,-7.466427,-7.523858,-29.490129,10.429112,-25.674637,-12.931184,...,0.396401,0.475632,0.486156,0.475511,0.444847,0.431168,0.396892,0.353266,0.316754,0.341479
304,/Users/ankitpokhrel/Desktop/infant_cry_detecti...,hungry,-66.182182,-11.319097,-17.508184,-30.855265,-23.461254,5.265229,-14.480108,-4.300076,...,0.417878,0.303164,0.280631,0.276635,0.306280,0.385472,0.429426,0.397041,0.325333,0.297114
408,/Users/ankitpokhrel/Desktop/infant_cry_detecti...,hungry,-340.684174,30.467274,-33.960754,-12.132354,4.591392,-9.283786,-20.571930,-10.817743,...,0.563271,0.567501,0.487104,0.484444,0.515815,0.420938,0.402524,0.502218,0.482425,0.444079
30,/Users/ankitpokhrel/Desktop/infant_cry_detecti...,discomfort,-92.714851,19.822575,-47.350857,-30.245199,-21.855501,-8.129131,-14.099007,-8.989731,...,0.240703,0.221219,0.220603,0.258790,0.316376,0.281375,0.352607,0.379621,0.356402,0.337036
454,/Users/ankitpokhrel/Desktop/infant_cry_detecti...,hungry,-285.942566,42.230091,-24.848104,-7.648776,-29.789537,-33.661163,-10.363840,-9.416376,...,0.220414,0.143338,0.121332,0.173716,0.324366,0.526013,0.448407,0.328658,0.221483,0.167091


In [18]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
final_df["label"] = le.fit_transform(final_df["label"])

In [19]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

X = final_df.drop(columns=["label", "audio_link"])
y = final_df["label"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [20]:
rf_model = RandomForestClassifier(n_estimators=100)
rf_model.fit(X_train, y_train)
y_pred = rf_model.predict(X_test)
print("Random Forest Accuracy:", accuracy_score(y_test, y_pred))

Random Forest Accuracy: 0.7934782608695652


In [21]:
xgb_model = XGBClassifier()
xgb_model.fit(X_train, y_train)
y_pred_xgb = xgb_model.predict(X_test)
print("XGBoost Accuracy:", accuracy_score(y_test, y_pred_xgb))

XGBoost Accuracy: 0.7934782608695652


In [22]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

model = keras.Sequential([
    layers.Dense(64, activation="relu", input_shape=(X_train.shape[1],)),
    layers.Dense(32, activation="relu"),
    layers.Dense(16, activation="relu"),
    layers.Dense(len(y.unique()), activation="softmax")
])

model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test))

loss, acc = model.evaluate(X_test, y_test)
print("Deep Learning Model Accuracy:", acc)

Epoch 1/50


/Users/ankitpokhrel/Desktop/infant_cry_detection/myenv/lib/python3.12/site-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.8350 - loss: 14.6185 - val_accuracy: 0.6630 - val_loss: 6.3206
Epoch 2/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6883 - loss: 5.6353 - val_accuracy: 0.7609 - val_loss: 4.2113
Epoch 3/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7001 - loss: 3.9877 - val_accuracy: 0.4783 - val_loss: 3.8498
Epoch 4/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6210 - loss: 2.8188 - val_accuracy: 0.7391 - val_loss: 2.7244
Epoch 5/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7091 - loss: 2.3420 - val_accuracy: 0.7065 - val_loss: 2.3457
Epoch 6/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7697 - loss: 1.9267 - val_accuracy: 0.7391 - val_loss: 2.0803
Epoch 7/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7315 - loss: 1.9476 - val_accuracy: 0.5217 - val_loss: 2.2797
Epoch 8/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7737 - loss: 1.6816 - val_accuracy: 0.7174 - val_loss: 2.1869
Ep

In [23]:
param_grid = {
    "n_estimators": [100, 200, 300],
    "max_depth": [10, 20, None],
    "min_samples_split": [2, 5, 10]
}

grid_search = GridSearchCV(RandomForestClassifier(), param_grid, cv=5, n_jobs=-1)
grid_search.fit(X_train, y_train)

print("Best Parameters:", grid_search.best_params_)
print("Best Accuracy:", grid_search.best_score_)

/opt/homebrew/Cellar/python@3.12/3.12.9/Frameworks/Python.framework/Versions/3.12/lib/python3.12/multiprocessing/queues.py:122: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  return _ForkingPickler.loads(res)
/opt/homebrew/Cellar/python@3.12/3.12.9/Frameworks/Python.framework/Versions/3.12/lib/python3.12/multiprocessing/queues.py:122: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  return _ForkingPickler.loads(res)
/opt/homebrew/Cellar/python@3.12/3.12.9/Frameworks/Python.framework/Versions/3.12/lib/python3.12/multiprocessing/queues.py:122: UserWarning: pkg_resources is deprecated as an API. See http

Best Parameters: {'max_depth': 10, 'min_samples_split': 2, 'n_estimators': 100}
Best Accuracy: 0.8465753424657534


In [24]:
def extract_audio_features(audio_path, n_mfcc=13):
    try:
        y, sr = librosa.load(audio_path, sr=None)
        mfccs = np.mean(librosa.feature.mfcc(y=y, sr=sr, n_mfcc=n_mfcc), axis=1)
        spectral_centroid = np.mean(librosa.feature.spectral_centroid(y=y, sr=sr), axis=1)
        zcr = np.mean(librosa.feature.zero_crossing_rate(y), axis=1)
        chroma = np.mean(librosa.feature.chroma_stft(y=y, sr=sr), axis=1)
        features = np.concatenate([
            mfccs, spectral_centroid.ravel(), zcr.ravel(), chroma
        ])
        return features
    except Exception as e:
        print(f"Error extracting features from {audio_path}: {e}")
        return None

In [25]:
X_new = extract_audio_features("/Users/ankitpokhrel/Desktop/infant_cry_detection/backend/model/donateacry_corpus/belly_pain/643D64AD-B711-469A-AF69-55C0D5D3E30F-1430138495-1.0-m-72-bp.wav")  # Function to extract MFCCs, Chroma, etc.
prediction = grid_search.predict([X_new])[0]
print(prediction)
print("Predicted Label:", le.inverse_transform([prediction]))

0
Predicted Label: ['belly_pain']


/Users/ankitpokhrel/Desktop/infant_cry_detection/myenv/lib/python3.12/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


In [26]:
import pickle
from sklearn.ensemble import RandomForestClassifier  # Example classifier

# Train your model
clf = RandomForestClassifier()
clf.fit(X_train, y_train)  # Ensure X_train and y_train are correctly defined

# Save the trained model, NOT features
with open("audio_prediction_model.pkl", "wb") as f:
    pickle.dump(clf, f)

print("Model saved successfully!")

Model saved successfully!
